Here we should make sure that our Transformer implementation works properly. For this purpose we gonna learn to translate on En-De pair

In [3]:
import sys; sys.path.append('..')

In [ ]:
import torch
import torch.nn as nn

Generating BPEs (joint for both langs)

In [21]:
%%bash
src="../data/multi30k/train.en"
trg="../data/multi30k/train.de"
num_bpes=16000

bpes="../data/generated/bpes"
src_vocab="../data/generated/vocab.en"
trg_vocab="../data/generated/vocab.de"

python ../ext-libs/subword-nmt/learn_joint_bpe_and_vocab.py \
    --input "$src" "$trg"
    -s "$num_bpes"
    -o "$bpes"
    --write-vocabulary "$src_vocab" "$trg_vocab"

# Now we should apply bpe for all our train/test/val files
for file in $(ls ../data/multi30k)
do
    #if [ ${file: -2} == "en" ]; then vocab="$src_vocab"; else vocab="$trg_vocab"; fi
    vocab="../data/generated/vocab.${file: -2}"
  
    python ../ext-libs/subword-nmt/apply_bpe.py -c $bpes \
       --vocabulary "$vocab" \
       < "../data/multi30k/$file" > "../data/generated/$file.bpe"
done